# The code here is to combine weather data and trip data

In [1]:
from datetime import datetime, date, time
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from ggplot import *

In [2]:
# lambdas & functions
lambda_city = lambda x: station_data['city'].ix[x]

def convert_float(val):
    try:
        return float(val)
    except ValueError:
        return 0

# dictionary reference
seasons_dict = {
    1: 'Winter',2: 'Spring',3: 'Spring',4: 'Spring',
    5: 'Summer',6: 'Summer',7: 'Summer',8: 'Autumn',
    9: 'Autumn',10: 'Autumn',11: 'Winter',12: 'Winter'}

city_dict = {
    94107: 'San Francisco',
    94063: 'Redwood City',
    94301: 'Palo Alto', 
    94041: 'Mountain View', 
    95113: 'San Jose'}

# header reference
station_headers = ['station_id','station_name','lat','long',
                   'dock_count','city','install_date']

status_headers = ['station_id','bikes_free','docks_free','time']

trip_headers = ['trip_id','trip_time',
                'start_dt','start_station','start_id',
                'end_dt','end_station','end_id',
                'bike_num','user_type','ZIP']

weather_headers = ['date',
                   'max_temp','mean_temp','min_temp',
                   'max_dp','mean_dp','min_dp',
                   'max_hum','mean_hum','min_hum',
                   'max_sea','mean_sea','min_sea',
                   'max_vis','mean_vis','min_vis',
                   'max_wind','mean_wind','max_gust',
                   'rain_inches','cloud_cover','events',
                   'wind_dir_degrees','ZIP']

## Weather data

In [24]:
# read weather data
weather_data = pd.read_csv('weather_data_new.csv', parse_dates=['date'])
weather_data['season'] = weather_data['date'].dt.month.map(seasons_dict)
weather_data['city'] = weather_data['ZIP'].map(city_dict)
# scrubbed columns from weather_data
weather_data['rain_inches'] = weather_data['rain_inches'].map(convert_float)

In [25]:
import datetime

In [31]:
# split weather data to 2014 and 2015
weather_2015 = weather_data[weather_data['date'] >='2014-09-01']
weather_2014 = weather_data[weather_data['date'] <'2014-09-01']

# label the datasets by month
weather_2014['month'] = weather_2014['date'].dt.month
weather_2015['month'] = weather_2015['date'].dt.month


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
# create a new dataframe that contains only the info that I need (month, season, avg_tem)
weather_2014_for_me = weather_2014[['date', 'mean_temp','mean_wind','season', 'city','month']]
weather_2015_for_me = weather_2015[['date', 'mean_temp','mean_wind','season', 'city','month']]

In [86]:
weather_2014_for_me['city'=='San Francisco']

,date,mean_temp,mean_wind,season,city,month
0,2013-09-01,66,13,Autumn,San Francisco,9
1,2013-09-01,70,4,Autumn,Mountain View,9
2,2013-09-01,71,7,Autumn,Palo Alto,9
3,2013-09-01,72,7,Autumn,San Jose,9
4,2013-09-01,71,5,Autumn,Redwood City,9


## Trip data

In [47]:
# read trip data and split it for 2014 and 2015
trip_data = pd.read_csv('trip_data_new.csv', parse_dates = ['date'])
trip_data_2014 = trip_data[trip_data['date'] < '2014-09-01']
trip_data_2015 = trip_data[trip_data['date'] >= '2014-09-01']

In [48]:
# label the trip datasets by month
trip_data_2014['month'] = trip_data_2014['date'].dt.month
trip_data_2015['month'] = trip_data_2015['date'].dt.month
trip_data_2014['trip_time'] = trip_data_2014['trip_time'] / 60   # concert trip time unit to minute
trip_data_2015['trip_time'] = trip_data_2015['trip_time'] / 60  

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [72]:
# create a new dataframe that contains only the info that I need (month, season, trip time, user type, city)
trip_data_2014_for_me = trip_data_2014[['date', 'trip_time','user_type','start_city', 'month']]
trip_data_2015_for_me = trip_data_2015[['date', 'trip_time','user_type','start_city','month']]

## Weather and trip data together per city

### San Francisco 2014 and 2015

In [126]:
# sf weather and trip data
SF_weather_2014 = weather_2014_for_me[weather_2014_for_me['city']=='San Francisco']
SF_trip_data_2014 = trip_data_2014_for_me[trip_data_2014_for_me['start_city'] =='San Francisco']

In [127]:
SF_all_2014 = pd.merge(SF_trip_data_2014, SF_weather_2014,on='date')
SF_all_2014.columns

Index(['date', 'trip_time', 'user_type', 'start_city', 'month_x', 'mean_temp',
       'mean_wind', 'season', 'city', 'month_y'],
      dtype='object')

In [128]:
SF_all_2014 = SF_all_2014.drop(['start_city', 'month_x'], axis=1)

In [143]:
# sf by user statistics
SF_all_2014.rename(columns={'month_y': 'month'})
SF_by_user_2014 = SF_all_2014.groupby('user_type')
#SF_by_user_2014_avg_trip_time = SF_by_user_2014.mean()['trip_time']
#SF_by_user_2014_avg_trip_time 

In [146]:
SF_by_user_2014.head()

,date,trip_time,user_type,mean_temp,mean_wind,season,city,month_y
0,2013-09-01,25.133333,Customer,66,13,Autumn,San Francisco,9
1,2013-09-01,4.566667,Subscriber,66,13,Autumn,San Francisco,9
2,2013-09-01,19.950000,Customer,66,13,Autumn,San Francisco,9
3,2013-09-01,25.183333,Customer,66,13,Autumn,San Francisco,9
4,2013-09-01,25.183333,Customer,66,13,Autumn,San Francisco,9
5,2013-09-01,13.966667,Customer,66,13,Autumn,San Francisco,9
19,2013-09-01,18.433333,Subscriber,66,13,Autumn,San Francisco,9
37,2013-09-01,17.300000,Subscriber,66,13,Autumn,San Francisco,9
40,2013-09-01,3.416667,Subscriber,66,13,Autumn,San Francisco,9
41,2013-09-01,14.333333,Subscriber,66,13,Autumn,San Francisco,9
